# Team Data Cleaning

Cleans efficiency rating datasets found at their respective websites and NCAA regular season and NCAA tournament data provided by Kaggle.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import re

## Team Efficiency Ratings

#### Colley Ratings

The first ratings are from http://www.colleyrankings.com/

(Note: Their archive did not contain data from the 2009 season)

In [2]:
colley = pd.read_csv('mydata/colley_pt.csv')

In [3]:
colley.head()

,rank,team,rating,season
0,1,UNC,1.08474,2008
1,2,MEMPHIS,1.04691,2008
2,3,UCLA,1.03847,2008
3,4,TENNESSEE,1.03505,2008
4,5,KANSAS,1.01374,2008


In [4]:
colley.tail()

,rank,team,rating,season
3475,347,NW St,0.042051,2018
3476,348,Bryant,0.024322,2018
3477,349,Miss Vly St,0.018847,2018
3478,350,Ala A&M,-0.015179,2018
3479,351,Delaware St,-0.056697,2018


In [5]:
# Put the team names in the same format (lowercase no punctuation) for joins later
colley['team'] = colley.team.str.replace('[^a-zA-Z&.() ]+',' ').str.lower()
colley['team'] = colley.team.str.replace('[^a-z& ]+', '')

#### Teamrank Ratings

The next ratings are from https://www.teamrankings.com/

The first dataset is their predictive ratings.
The second dataset is their power rankings over the last 10 games.

In [6]:
teamrank = pd.read_csv('mydata/teamrank_pt.csv')
teamrank10 = pd.read_csv('mydata/teamrank10_pt.csv')

In [7]:
teamrank.head()

,Rank,Team,Rating,Season
0,1,Kansas (31-3),32.4,2008
1,2,N Carolina (32-2),29.4,2008
2,3,Memphis (33-1),28.7,2008
3,4,Duke (27-5),28.5,2008
4,5,UCLA (31-3),28.2,2008


In [8]:
teamrank.tail()

,Rank,Team,Rating,Season
4761,347,Chicago St (3-29),-17.6,2018
4762,348,Miss Val St (4-28),-18.1,2018
4763,349,Maryland ES (7-25),-19.2,2018
4764,350,Delaware St (4-28),-19.9,2018
4765,351,Alab A&M (3-28),-20.3,2018


In [9]:
teamrank10.head()

,Rank,Team,Rating,Season
0,1,Duke (10-0),33.9,2008
1,2,W Virginia (9-1),32.6,2008
2,3,Kansas (10-0),31.8,2008
3,4,UCLA (9-1),29.3,2008
4,5,N Carolina (10-0),28.7,2008


In [10]:
teamrank10.tail()

,Rank,Team,Rating,Season
4209,347,NW State (1-9),-18.8,2018
4210,348,Bryant (1-9),-19.3,2018
4211,349,Delaware St (2-8),-19.9,2018
4212,350,Maryland ES (2-8),-22.8,2018
4213,351,Alab A&M (1-9),-27.2,2018


The Teamrank datasets contain some observations from non-Division 1 teams. The code below removes those teams by filtering out teams with less than 20 games played at the Division 1 level.

In [11]:
# Removes teams that are not in Division 1
to_drop = []
for i in range(len(teamrank.Team)):
    name = teamrank.iloc[i].Team
    num_games = 0
    
    # Loops over every character of the string in the Team column
    for j in range(len(name)):
        if name[j] == '(' and name[j + 1].isdigit():   # The start of the W-L record, wins are the next number
            games_first_digit = int(name[j + 1])
            games_second_digit = 0
            
            # If the number of wins is in the double digits
            if name[j + 2] != '-':
                games_second_digit = int(name[j + 2])
                games_first_digit = games_first_digit * 10
                
            num_games = num_games + games_first_digit + games_second_digit
            
        if name[j] == '-' and name[j + 1].isdigit():   # The losses are the next number
            games_first_digit = int(name[j + 1])
            games_second_digit = 0
            
            # If the number of losses is in the double digits
            if name[j + 2] != ')':
                games_second_digit = int(name[j + 2])
                games_first_digit = games_first_digit * 10
                
            num_games = num_games + games_first_digit + games_second_digit
            
    # Division 1 teams always play more than 20 games, while non-Division 1 always do not
    if num_games < 20:
        to_drop.append(i)
        
teamrank = teamrank.drop(to_drop)

In [12]:
# Removes teams that are not in Division 1
to_drop = []
for i in range(len(teamrank10.Team)):
    name = teamrank10.iloc[i].Team
    num_games = 0
    
    # Loops over every character of the string in the Team column
    for j in range(len(name)):
        if name[j] == '(' and name[j + 1].isdigit():   # The start of the W-L record, wins are the next number
            games_first_digit = int(name[j + 1])
            games_second_digit = 0
            
            # If the number of wins is in the double digits
            if name[j + 2] != '-':
                games_second_digit = int(name[j + 2])
                games_first_digit = games_first_digit * 10
                
            num_games = num_games + games_first_digit + games_second_digit
         
        
        if name[j] == '-' and name[j + 1].isdigit():    # The losses are the next number
            games_first_digit = int(name[j + 1])
            games_second_digit = 0
            
            # If the number of losses is in the double digits
            if name[j + 2] != ')':
                games_second_digit = int(name[j + 2])
                games_first_digit = games_first_digit * 10
                
            num_games = num_games + games_first_digit + games_second_digit
            
    # If the record over the last 10 games does not add up to 10, the team is not in Division 1
    if num_games < 10:
        to_drop.append(i)
        
teamrank10 = teamrank10.drop(to_drop)

In [13]:
# Put the team names in the same format (lowercase no punctuation) for joins later
teamrank['Team'] = teamrank['Team'].str.replace('[^a-zA-Z&.() ]+', ' ').str.lower()
teamrank['Team'] = teamrank['Team'].str.replace('[^a-z& ]+', '').str.rstrip()
teamrank10['Team'] = teamrank10['Team'].str.replace('[^a-zA-Z&.() ]+', ' ').str.lower()
teamrank10['Team'] = teamrank10['Team'].str.replace('[^a-z& ]+', '').str.rstrip()

In [14]:
teamrank.head()

,Rank,Team,Rating,Season
0,1,kansas,32.4,2008
1,2,n carolina,29.4,2008
2,3,memphis,28.7,2008
3,4,duke,28.5,2008
4,5,ucla,28.2,2008


In [15]:
teamrank10.head()

,Rank,Team,Rating,Season
0,1,duke,33.9,2008
1,2,w virginia,32.6,2008
2,3,kansas,31.8,2008
3,4,ucla,29.3,2008
4,5,n carolina,28.7,2008


#### Trank Ratings

The next ratings from http://www.barttorvik.com/#

(Note: This data only goes back to 2008, so that is why every dataset starts at 2008)

(Note: Their data also contains some offensive and defensive team stats)

In [16]:
trank = pd.read_csv('mydata/trank_pt.csv')

In [17]:
trank.head()

,RK,TEAM,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,TOR,TORD,ORB,DRB,FTR,FTRD,2P%,2P%D,3P%,3P%D,ADJ T.,WAB,Season
0,1,Kansas,121.0,85.6,0.9816,56.3,44.8,18.7,22.9,38.0,29.0,37.5,30.8,54.8,40.9,39.9,34.0,69.5,9.9,2008
1,2,Memphis,113.3,83.8,0.9697,53.0,42.5,17.2,23.3,37.8,29.3,38.6,32.7,53.3,41.6,35.0,29.8,70.7,8.9,2008
2,3,UCLA,116.4,86.9,0.9664,52.3,48.0,18.8,22.7,39.2,25.9,38.0,25.7,52.9,46.8,33.8,34.2,66.2,10.8,2008
3,4,Wisconsin,112.2,84.9,0.9609,50.6,43.1,19.4,22.2,36.2,28.5,39.3,25.4,49.2,41.3,35.9,31.1,63.5,8.3,2008
4,5,Duke,117.2,88.8,0.9607,54.1,47.6,18.2,24.9,34.3,33.5,40.4,32.0,51.8,47.1,38.4,32.8,73.7,8.9,2008


In [18]:
trank.tail()

,RK,TEAM,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,TOR,TORD,ORB,DRB,FTR,FTRD,2P%,2P%D,3P%,3P%D,ADJ T.,WAB,Season
3819,347,Coppin St.,85.9,109.6,0.0574,43.3,49.2,20.5,17.9,22.3,31.9,27.1,37.6,41.0,48.3,30.5,33.7,68.7,-20.9,2018
3820,348,Mississippi Valley St.,89.2,114.8,0.0520,42.6,54.9,19.1,19.6,30.4,31.1,29.1,39.2,41.6,54.6,30.1,36.8,69.9,-22.5,2018
3821,349,Maryland Eastern Shore,90.9,117.7,0.0489,43.1,54.7,19.4,16.0,28.2,34.7,29.7,31.8,44.3,54.3,27.5,36.8,66.4,-19.3,2018
3822,350,Alabama A&M,87.4,114.1,0.0446,45.0,54.0,24.2,16.6,30.0,32.5,31.4,29.8,44.8,52.3,30.3,38.4,67.5,-23.0,2018
3823,351,Delaware St.,90.3,119.9,0.0368,47.3,55.9,22.8,18.2,29.3,33.7,28.5,35.7,46.1,53.8,33.2,38.9,66.2,-24.1,2018


In [19]:
# Put the team names in the same format (lowercase no punctuation) for joins later
trank['TEAM'] = trank.TEAM.str.replace('[^a-zA-Z&.() ]+',' ').str.lower()
trank['TEAM'] = trank.TEAM.str.replace('[^a-z& ]+','').str.rstrip()

#### Kenpom Ratings

The next ratings are from https://kenpom.com/

In [20]:
kenpom_08 = pd.read_csv('mydata/summary08_pt.csv')
kenpom_09 = pd.read_csv('mydata/summary09_pt.csv')
kenpom_10 = pd.read_csv('mydata/summary10_pt.csv')
kenpom_11 = pd.read_csv('mydata/summary11_pt.csv')
kenpom_12 = pd.read_csv('mydata/summary12_pt.csv')
kenpom_13 = pd.read_csv('mydata/summary13_pt.csv')
kenpom_14 = pd.read_csv('mydata/summary14_pt.csv')
kenpom_15 = pd.read_csv('mydata/summary15_pt.csv')
kenpom_16 = pd.read_csv('mydata/summary16_pt.csv')
kenpom_17 = pd.read_csv('mydata/summary17_pt.csv')
kenpom_18 = pd.read_csv('mydata/summary18_pt.csv')

In [21]:
kenpom_08.head()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
0,2008,Kansas,68.8789,106,66.8546,88,117.530,1,121.433,1,89.3201,4,87.4681,3,33.9647,1,1.0
1,2008,UCLA,65.0976,246,63.1761,220,113.555,10,118.735,6,91.6434,8,88.1600,4,30.5753,2,1.0
2,2008,Memphis,70.5062,59,67.7113,63,112.778,18,114.695,25,85.6765,1,85.3169,1,29.3779,3,1.0
3,2008,North Carolina,76.0195,3,71.9191,8,115.213,4,120.686,2,94.7521,33,92.5415,20,28.1446,4,1.0
4,2008,Duke,74.3046,12,70.9709,12,112.963,15,117.443,8,93.0572,20,89.8149,7,27.6285,5,2.0


In [22]:
kenpom_18.tail()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
346,2018,Wright St.,69.7118,157,68.2124,175,101.383,238,101.068,248,94.0649,9,98.2510,53,2.81674,135,14.0
347,2018,Wyoming,74.6974,13,73.5044,11,103.046,197,105.921,156,102.0530,122,100.8070,89,5.11463,115,NaN
348,2018,Xavier,72.1555,51,70.4307,61,115.530,13,120.740,7,102.2090,128,99.0492,59,21.69090,14,1.0
349,2018,Yale,70.0380,136,68.7887,138,104.047,168,104.050,191,106.1810,237,107.2290,211,-3.17937,197,NaN
350,2018,Youngstown St.,73.0127,28,71.0824,49,99.086,276,101.405,244,113.7210,336,116.4890,338,-15.08350,325,NaN


In [23]:
kenpom_seasons = [kenpom_08, kenpom_09, kenpom_10, kenpom_11, kenpom_12, kenpom_13, kenpom_14, kenpom_15, kenpom_16, kenpom_17, kenpom_18]
kenpom = pd.concat(kenpom_seasons)

In [24]:
kenpom.head()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
0,2008,Kansas,68.8789,106,66.8546,88,117.530,1,121.433,1,89.3201,4,87.4681,3,33.9647,1,1.0
1,2008,UCLA,65.0976,246,63.1761,220,113.555,10,118.735,6,91.6434,8,88.1600,4,30.5753,2,1.0
2,2008,Memphis,70.5062,59,67.7113,63,112.778,18,114.695,25,85.6765,1,85.3169,1,29.3779,3,1.0
3,2008,North Carolina,76.0195,3,71.9191,8,115.213,4,120.686,2,94.7521,33,92.5415,20,28.1446,4,1.0
4,2008,Duke,74.3046,12,70.9709,12,112.963,15,117.443,8,93.0572,20,89.8149,7,27.6285,5,2.0


In [25]:
kenpom.tail()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed
346,2018,Wright St.,69.7118,157,68.2124,175,101.383,238,101.068,248,94.0649,9,98.2510,53,2.81674,135,14.0
347,2018,Wyoming,74.6974,13,73.5044,11,103.046,197,105.921,156,102.0530,122,100.8070,89,5.11463,115,NaN
348,2018,Xavier,72.1555,51,70.4307,61,115.530,13,120.740,7,102.2090,128,99.0492,59,21.69090,14,1.0
349,2018,Yale,70.0380,136,68.7887,138,104.047,168,104.050,191,106.1810,237,107.2290,211,-3.17937,197,NaN
350,2018,Youngstown St.,73.0127,28,71.0824,49,99.086,276,101.405,244,113.7210,336,116.4890,338,-15.08350,325,NaN


In [26]:
# Put the team names in the same format (lowercase no punctuation) for joins later
kenpom['TeamName'] = kenpom.TeamName.str.replace('[^a-zA-Z&.() ]+',' ').str.lower()
kenpom['TeamName'] = kenpom.TeamName.str.replace('[^a-z& ]+','')

## Joining the Datasets

The next csv file was provided by Kaggle for their competition. It contains a TeamID for each Division 1 team.

In [27]:
spellings = pd.read_csv('DataFiles/TeamSpellings.csv', encoding = "ISO-8859-1")

In [28]:
spellings.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


In [29]:
# Put the team names in the same format (lowercase no punctuation) for joins later
spellings['TeamNameSpelling'] = spellings['TeamNameSpelling'].str.replace('[^a-z&. ]+', ' ')
spellings['TeamNameSpelling'] = spellings['TeamNameSpelling'].str.replace('[^a-z& ]+', '')

Now that I have all of the data, I can begin to join each ratings dataset with the team spellings to associate the data for each team with their unique TeamID.

In [30]:
colley_teams = pd.merge(colley, spellings, how = 'left', left_on = 'team', right_on = 'TeamNameSpelling')

In [31]:
nans = lambda df: df[df.isnull().any(axis=1)]  # Function to print out rows with null values
nans(colley_teams)

,rank,team,rating,season,TeamNameSpelling,TeamID
17,18,mich st,0.881696,2008,NaN,NaN
22,22,wash st,0.861280,2008,NaN,NaN
26,25,ill st,0.825792,2008,NaN,NaN
27,26,wvu,0.821954,2008,NaN,NaN
41,39,miss st,0.788609,2008,NaN,NaN
43,41,s alabama,0.783740,2008,NaN,NaN
69,59,ariz st,0.728490,2008,NaN,NaN
81,67,oral rob,0.709561,2008,NaN,NaN
83,69,rh island,0.702097,2008,NaN,NaN
84,70,geo mason,0.698031,2008,NaN,NaN


Many of the team names in the Colley Ratings were different from the names in the Kaggle Dataset, therefore I must change the names in the Colley Ratings so I can merge the two datasets together.

In [32]:
# For loop to change team names above
for i in range(len(colley.team)):
    if colley.iloc[i, 1] == 'texas pan am' or colley.iloc[i ,1] == 'tx pan am':
        colley.iloc[i, 1] = 'texas pan american'
    if colley.iloc[i, 1] == 'j madison':
        colley.iloc[i, 1] = 'james madison'
    if colley.iloc[i, 1] == 'wi grn bay':
        colley.iloc[i, 1] = 'green bay'
    if colley.iloc[i, 1] == 'la lafay':
        colley.iloc[i, 1] = 'louisiana lafayette'
    if colley.iloc[i, 1] == 'will & mary':
        colley.iloc[i, 1] = 'william & mary'
    if colley.iloc[i, 1] == 'cs nrthrdg':
        colley.iloc[i, 1] = 'cal st northridge'
    if colley.iloc[i, 1] == 'tx san ant':
        colley.iloc[i, 1] = 'utsa'
    if colley.iloc[i, 1] == 'utah vly st':
        colley.iloc[i, 1] = 'utah valley st'
    if colley.iloc[i, 1] == 'cal san fr':
        colley.iloc[i, 1] = 'san francisco'
    if colley.iloc[i, 1] == 'cincy':
        colley.iloc[i, 1] = 'cincinnati'
    if colley.iloc[i, 1] == 'tex tech':
        colley.iloc[i, 1] = 'texas tech'
    if colley.iloc[i, 1] == 'wvu':
        colley.iloc[i, 1] = 'west virginia'
    if colley.iloc[i, 1] == 'mich st':
        colley.iloc[i, 1] = 'michigan st'
    if colley.iloc[i, 1] == 'rh island':
        colley.iloc[i, 1] = 'rhode island'
    if colley.iloc[i, 1] == 'loyola chi':
        colley.iloc[i, 1] = 'loyola chicago'
    if colley.iloc[i, 1] == 'st bonny':
        colley.iloc[i, 1] = 'st bonaventure'
    if colley.iloc[i, 1] == 'miss st':
        colley.iloc[i, 1] = 'mississippi st'
    if colley.iloc[i, 1] == 'new mex st':
        colley.iloc[i, 1] = 'new mexico st'
    if colley.iloc[i, 1] == 'ariz st':
        colley.iloc[i, 1] = 'arizona st'
    if colley.iloc[i, 1] == 'okl st':
        colley.iloc[i, 1] = 'oklahoma st'
    if colley.iloc[i, 1] == 'odu':
        colley.iloc[i, 1] = 'old dominion'
    if colley.iloc[i, 1] == 'unc grnsbro':
        colley.iloc[i, 1] = 'unc greensboro'
    if colley.iloc[i, 1] == 'col chrlstn':
        colley.iloc[i, 1] = 'college of charleston'
    if colley.iloc[i, 1] == 'csu sacramento':
        colley.iloc[i, 1] = 'sacramento st'
    if colley.iloc[i, 1] == 'boston coll':
        colley.iloc[i, 1] = 'boston college'
    if colley.iloc[i, 1] == 's carolina':
        colley.iloc[i, 1] = 'south carolina'
    if colley.iloc[i, 1] == 'n eastern':
        colley.iloc[i, 1] = 'northeastern'
    if colley.iloc[i, 1] == 'ill st':
        colley.iloc[i, 1] = 'illinois st'
    if colley.iloc[i, 1] == 'east mich':
        colley.iloc[i, 1] = 'eastern michigan'
    if colley.iloc[i, 1] == 'ga st':
        colley.iloc[i, 1] = 'georgia st'
    if colley.iloc[i, 1] == 'cs full':
        colley.iloc[i, 1] = 'cal st fullerton'
    if colley.iloc[i, 1] == 'n iowa':
        colley.iloc[i, 1] = 'northern iowa'
    if colley.iloc[i, 1] == 'ga tech':
        colley.iloc[i, 1] = 'georgia tech'
    if colley.iloc[i, 1] == 'jax st':
        colley.iloc[i, 1] = 'jacksonville st'
    if colley.iloc[i, 1] == 'n western':
        colley.iloc[i, 1] = 'northwestern'
    if colley.iloc[i, 1] == 'tx arl':
        colley.iloc[i, 1] = 'ut arlington'
    if colley.iloc[i, 1] == 'unc ash':
        colley.iloc[i, 1] = 'unc asheville'
    if colley.iloc[i, 1] == 'ga sou':
        colley.iloc[i, 1] = 'georgia southern'
    if colley.iloc[i, 1] == 'cent mich':
        colley.iloc[i, 1] = 'central michigan'
    if colley.iloc[i, 1] == 'wash st':
        colley.iloc[i, 1] = 'washington st'
    if colley.iloc[i, 1] == 'e wash':
        colley.iloc[i, 1] = 'eastern washington'
    if colley.iloc[i, 1] == 'west mich':
        colley.iloc[i, 1] = 'western michigan'
    if colley.iloc[i, 1] == 'ca irvine':
        colley.iloc[i, 1] = 'uc irvine'
    if colley.iloc[i, 1] == 'geo wash':
        colley.iloc[i, 1] = 'george washington'
    if colley.iloc[i, 1] == 'tenn tech':
        colley.iloc[i, 1] = 'tennessee tech'
    if colley.iloc[i, 1] == 'se la':
        colley.iloc[i, 1] = 'se louisiana'
    if colley.iloc[i, 1] == 'valpo':
        colley.iloc[i, 1] = 'valparaiso'
    if colley.iloc[i, 1] == 'geo mason':
        colley.iloc[i, 1] = 'george mason'
    if colley.iloc[i, 1] == 'la tech':
        colley.iloc[i, 1] = 'louisiana tech'
    if colley.iloc[i, 1] == 'bowling grn':
        colley.iloc[i, 1] = 'bowling green'
    if colley.iloc[i, 1] == 'l beach st':
        colley.iloc[i, 1] = 'long beach st'
    if colley.iloc[i, 1] == 'tenn st':
        colley.iloc[i, 1] = 'tennessee st'
    if colley.iloc[i, 1] == 'n texas':
        colley.iloc[i, 1] = 'north texas'
    if colley.iloc[i, 1] == 'sam hstn st':
        colley.iloc[i, 1] = 'sam houston st'
    if colley.iloc[i, 1] == 's miss':
        colley.iloc[i, 1] = 'southern miss'
    if colley.iloc[i, 1] == 'st fran pa':
        colley.iloc[i, 1] = 'st francis pa'
    if colley.iloc[i, 1] == 'wi milw':
        colley.iloc[i, 1] = 'milwaukee'
    if colley.iloc[i, 1] == 'r morris':
        colley.iloc[i, 1] = 'robert morris'
    if colley.iloc[i, 1] == 'ecu':
        colley.iloc[i, 1] = 'east carolina'
    if colley.iloc[i, 1] == 'tx sou':
        colley.iloc[i, 1] = 'texas southern'
    if colley.iloc[i, 1] == 'loy marymnt':
        colley.iloc[i, 1] = 'loyola marymount'
    if colley.iloc[i, 1] == 's utah':
        colley.iloc[i, 1] = 'southern utah'
    if colley.iloc[i, 1] == 'appy st':
        colley.iloc[i, 1] = 'appalachian st'
    if colley.iloc[i, 1] == 'se missouri':
        colley.iloc[i, 1] = 'se missouri st'
    if colley.iloc[i, 1] == 'stony brk':
        colley.iloc[i, 1] = 'stony brook'
    if colley.iloc[i, 1] == 's florida':
        colley.iloc[i, 1] = 'south florida'
    if colley.iloc[i, 1] == 'gard webb':
        colley.iloc[i, 1] = 'gardner webb'
    if colley.iloc[i, 1] == 'unc wilm':
        colley.iloc[i, 1] = 'unc wilmington'
    if colley.iloc[i, 1] == 'beth cook':
        colley.iloc[i, 1] = 'bethune cookman'
    if colley.iloc[i, 1] == 'fl intl':
        colley.iloc[i, 1] = 'florida international'
    if colley.iloc[i, 1] == 'oral rob':
        colley.iloc[i, 1] = 'oral roberts'
    if colley.iloc[i, 1] == 's alabama':
        colley.iloc[i, 1] = 'south alabama'
    if colley.iloc[i, 1] == 'chrlstn sou':
        colley.iloc[i, 1] = 'charleston southern'
    if colley.iloc[i, 1] == 'clvlnd st':
        colley.iloc[i, 1] = 'cleveland st'
    if colley.iloc[i, 1] == 'fairl dcknsn':
        colley.iloc[i, 1] = 'fairleigh dickinson'
    if colley.iloc[i, 1] == 'jax':
        colley.iloc[i, 1] = 'jacksonville'
    if colley.iloc[i, 1] == 'cent conn st':
        colley.iloc[i, 1] = 'central conn st'
    if colley.iloc[i, 1] == 'west ill':
        colley.iloc[i, 1] = 'western illinois'
    if colley.iloc[i, 1] == 'ark st':
        colley.iloc[i, 1] = 'arkansas st'
    if colley.iloc[i, 1] == 'ut chat':
        colley.iloc[i, 1] = 'chattanooga'
    if colley.iloc[i, 1] == 'uc riv':
        colley.iloc[i, 1] = 'cal riverside'
    if colley.iloc[i, 1] == 'st fran ny':
        colley.iloc[i, 1] = 'st francis ny'
    if colley.iloc[i, 1] == 'wisc green bay':
        colley.iloc[i, 1] = 'green bay'
    if colley.iloc[i, 1] == 'yngstwn st':
        colley.iloc[i, 1] = 'youngstown st'
    if colley.iloc[i, 1] == 'ark l rock':
        colley.iloc[i, 1] = 'arkansas little rock'
    if colley.iloc[i, 1] == 'tx a&m cc':
        colley.iloc[i, 1] = 'a&m corpus chris'
    if colley.iloc[i, 1] == 'new hamp':
        colley.iloc[i, 1] = 'new hampshire'
    if colley.iloc[i, 1] == 'ar pine bluff':
        colley.iloc[i, 1] = 'arkansas pine bluff'
    if colley.iloc[i, 1] == 'ala a&m':
        colley.iloc[i, 1] = 'alabama a&m'
    if colley.iloc[i, 1] == 'miss vly st':
        colley.iloc[i, 1] = 'ms valley st'
    if colley.iloc[i, 1] == 'nw st':
        colley.iloc[i, 1] = 'northwestern st'
    if colley.iloc[i, 1] == 'ala st':
        colley.iloc[i, 1] = 'alabama st'
    if colley.iloc[i, 1] == 'sc st':
        colley.iloc[i, 1] = 'south carolina st'
    if colley.iloc[i, 1] == 'n arizona':
        colley.iloc[i, 1] = 'northern arizona'

In [33]:
colley_teams = pd.merge(colley, spellings, how = 'left', left_on = 'team', right_on = 'TeamNameSpelling')
nans(colley_teams)

,rank,team,rating,season,TeamNameSpelling,TeamID


Now the datasets were joined perfectly.

In [34]:
# Need to drop duplicates because the spellings dataset had multiple rows for each team
colley_teams = colley_teams.drop(columns = ['rank', 'TeamNameSpelling', 'team']).drop_duplicates()  
colley_teams = colley_teams.rename(index = str, columns = {'rating': 'Colley_Rating', 'season': 'Season'})

In [35]:
colley_teams.head()

,Colley_Rating,Season,TeamID
0,1.08474,2008,1314
1,1.04691,2008,1272
2,1.03847,2008,1417
3,1.03505,2008,1397
4,1.01374,2008,1242


In [36]:
colley_teams.tail()

,Colley_Rating,Season,TeamID
5537,0.042051,2018,1322
5539,0.024322,2018,1136
5540,0.018847,2018,1290
5541,-0.015179,2018,1105
5542,-0.056697,2018,1175


In [37]:
teamrank_teams = pd.merge(teamrank, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(teamrank_teams)

,Rank,Team,Rating,Season,TeamNameSpelling,TeamID
1,2,n carolina,29.4,2008,NaN,NaN
12,13,w virginia,23.4,2008,NaN,NaN
15,16,wash state,22.7,2008,NaN,NaN
39,33,miss state,19.4,2008,NaN,NaN
40,34,st marys,19.3,2008,NaN,NaN
54,47,ga tech,17.4,2008,NaN,NaN
55,48,va tech,17.4,2008,NaN,NaN
58,51,st josephs,16.9,2008,NaN,NaN
75,62,u mass,15.9,2008,NaN,NaN
92,74,s alabama,14.8,2008,NaN,NaN


Many of the team names in the Teamrank Ratings were different from the names in the Kaggle Dataset, therefore I must change the names in the Teamrank Ratings so I can merge the two datasets together.

In [38]:
# For loop to change team names above
for i in range(len(teamrank.Team)):
    if teamrank.iloc[i, 1] == 'st marys':
        teamrank.iloc[i, 1] = 'st marys ca'
    if teamrank.iloc[i, 1] == 'wins salem':
        teamrank.iloc[i, 1] = 'winston salem'
    if teamrank.iloc[i, 1] == 'w virginia':
        teamrank.iloc[i, 1] = 'west virginia'
    if teamrank.iloc[i, 1] == 'n carolina':
        teamrank.iloc[i, 1] = 'north carolina'
    if teamrank.iloc[i, 1] == 'tx christian':
        teamrank.iloc[i, 1] = 'tcu'
    if teamrank.iloc[i, 1] == 'va tech':
        teamrank.iloc[i, 1] = 'virginia tech'
    if teamrank.iloc[i, 1] == 'miss state':
        teamrank.iloc[i, 1] = 'mississippi st'
    if teamrank.iloc[i, 1] == 'st bonavent':
        teamrank.iloc[i, 1] = 'st bonaventure'
    if teamrank.iloc[i, 1] == 'loyola chi':
        teamrank.iloc[i, 1] = 'loyola chicago'
    if teamrank.iloc[i, 1] == 's methodist':
        teamrank.iloc[i, 1] = 'smu'
    if teamrank.iloc[i, 1] == 'n mex state':
        teamrank.iloc[i, 1] = 'new mexico st'
    if teamrank.iloc[i, 1] == 's carolina':
        teamrank.iloc[i, 1] = 'south carolina'
    if teamrank.iloc[i, 1] == 'boston col':
        teamrank.iloc[i, 1] = 'boston college'
    if teamrank.iloc[i, 1] == 'e tenn st':
        teamrank.iloc[i, 1] = 'etsu'
    if teamrank.iloc[i, 1] == 'nc grnsboro':
        teamrank.iloc[i, 1] = 'unc greensboro'
    if teamrank.iloc[i, 1] == 'central fl':
        teamrank.iloc[i, 1] = 'ucf'
    if teamrank.iloc[i, 1] == 'utah val st':
        teamrank.iloc[i, 1] = 'utah valley st'
    if teamrank.iloc[i, 1] == 'northeastrn':
        teamrank.iloc[i, 1] = 'northeastern'
    if teamrank.iloc[i, 1] == 'ga tech':
        teamrank.iloc[i, 1] = 'georgia tech'
    if teamrank.iloc[i, 1] == 'col charlestn':
        teamrank.iloc[i, 1] = 'college of charleston'
    if teamrank.iloc[i, 1] == 'st josephs':
        teamrank.iloc[i, 1] = 'st josephs pa'
    if teamrank.iloc[i, 1] == 'u penn':
        teamrank.iloc[i, 1] = 'penn'
    if teamrank.iloc[i, 1] == 'ste f austin':
        teamrank.iloc[i, 1] = 'stephen f austin'
    if teamrank.iloc[i, 1] == 'fla gulf cst':
        teamrank.iloc[i, 1] = 'florida gulf coast'
    if teamrank.iloc[i, 1] == 'grd canyon':
        teamrank.iloc[i, 1] = 'grand canyon'
    if teamrank.iloc[i, 1] == 'tx arlington':
        teamrank.iloc[i, 1] = 'ut arlington'
    if teamrank.iloc[i, 1] == 'n iowa':
        teamrank.iloc[i, 1] = 'northern iowa'
    if teamrank.iloc[i, 1] == 'la tech':
        teamrank.iloc[i, 1] = 'louisiana tech'
    if teamrank.iloc[i, 1] == 'wm & mary':
        teamrank.iloc[i, 1] = 'william & mary'
    if teamrank.iloc[i, 1] == 'jksnville st':
        teamrank.iloc[i, 1] = 'jacksonville st'
    if teamrank.iloc[i, 1] == 'app state':
        teamrank.iloc[i, 1] = 'appalachian st'
    if teamrank.iloc[i, 1] == 'san fransco':
        teamrank.iloc[i, 1] = 'san francisco'
    if teamrank.iloc[i, 1] == 'e washingtn':
        teamrank.iloc[i, 1] = 'eastern washington'
    if teamrank.iloc[i, 1] == 'geo wshgtn':
        teamrank.iloc[i, 1] = 'george washington'
    if teamrank.iloc[i, 1] == 'u mass':
        teamrank.iloc[i, 1] = 'umass'
    if teamrank.iloc[i, 1] == 'maryland bc':
        teamrank.iloc[i, 1] = 'umbc'
    if teamrank.iloc[i, 1] == 'wash state':
        teamrank.iloc[i, 1] = 'washington st'
    if teamrank.iloc[i, 1] == 'tx san ant':
        teamrank.iloc[i, 1] = 'utsa'
    if teamrank.iloc[i, 1] == 'st fran pa':
        teamrank.iloc[i, 1] = 'st francis pa'
    if teamrank.iloc[i, 1] == 'miami oh':
        teamrank.iloc[i, 1] = 'miami ohio'
    if teamrank.iloc[i, 1] == 'geo mason':
        teamrank.iloc[i, 1] = 'george mason'
    if teamrank.iloc[i, 1] == 'wi milwkee':
        teamrank.iloc[i, 1] = 'milwaukee'
    if teamrank.iloc[i, 1] == 'tn state':
        teamrank.iloc[i, 1] = 'tennessee st'
    if teamrank.iloc[i, 1] == 'tn tech':
        teamrank.iloc[i, 1] = 'tennessee tech'
    if teamrank.iloc[i, 1] == 'nc wilmgton':
        teamrank.iloc[i, 1] = 'unc wilmington'
    if teamrank.iloc[i, 1] == 's alabama':
        teamrank.iloc[i, 1] = 'south alabama'
    if teamrank.iloc[i, 1] == 'lg beach st':
        teamrank.iloc[i, 1] = 'long beach st'
    if teamrank.iloc[i, 1] == 'james mad':
        teamrank.iloc[i, 1] = 'james madison'
    if teamrank.iloc[i, 1] == 'sam hous st':
        teamrank.iloc[i, 1] = 'sam houston st'
    if teamrank.iloc[i, 1] == 'cs bakersfld':
        teamrank.iloc[i, 1] = 'cal state bakersfield'
    if teamrank.iloc[i, 1] == 'loyola mymt':
        teamrank.iloc[i, 1] = 'loyola marymount'
    if teamrank.iloc[i, 1] == 's mississippi':
        teamrank.iloc[i, 1] = 'southern miss'
    if teamrank.iloc[i, 1] == 'bowling grn':
        teamrank.iloc[i, 1] = 'bowling green'
    if teamrank.iloc[i, 1] == 'tx el paso':
        teamrank.iloc[i, 1] = 'utep'
    if teamrank.iloc[i, 1] == 'n hampshire':
        teamrank.iloc[i, 1] = 'new hampshire'
    if teamrank.iloc[i, 1] == 'rob morris':
        teamrank.iloc[i, 1] = 'robert morris'
    if teamrank.iloc[i, 1] == 'wi grn bay':
        teamrank.iloc[i, 1] = 'green bay'
    if teamrank.iloc[i, 1] == 'charl south':
        teamrank.iloc[i, 1] = 'charleston southern'
    if teamrank.iloc[i, 1] == 'abl christian':
        teamrank.iloc[i, 1] = 'abilene christian'
    if teamrank.iloc[i, 1] == 'gard webb':
        teamrank.iloc[i, 1] = 'gardner webb'
    if teamrank.iloc[i, 1] == 'tx pan am':
        teamrank.iloc[i, 1] = 'texas pan american'
    if teamrank.iloc[i, 1] == 'se missouri':
        teamrank.iloc[i, 1] = 'se missouri st'
    if teamrank.iloc[i, 1] == 'neb omaha':
        teamrank.iloc[i, 1] = 'omaha'
    if teamrank.iloc[i, 1] == 's florida':
        teamrank.iloc[i, 1] = 'south florida'
    if teamrank.iloc[i, 1] == 'mass lowell':
        teamrank.iloc[i, 1] = 'umass lowell'
    if teamrank.iloc[i, 1] == 'e carolina':
        teamrank.iloc[i, 1] = 'east carolina'
    if teamrank.iloc[i, 1] == 'tx a&m cc':
        teamrank.iloc[i, 1] = 'a&m corpus chris'
    if teamrank.iloc[i, 1] == 's utah':
        teamrank.iloc[i, 1] = 'southern utah'
    if teamrank.iloc[i, 1] == 'n florida':
        teamrank.iloc[i, 1] = 'north florida'
    if teamrank.iloc[i, 1] == 'sacred hrt':
        teamrank.iloc[i, 1] = 'sacred heart'
    if teamrank.iloc[i, 1] == 'st fran ny':
        teamrank.iloc[i, 1] = 'st francis ny'
    if teamrank.iloc[i, 1] == 'ar lit rock':
        teamrank.iloc[i, 1] = 'arkansas little rock'
    if teamrank.iloc[i, 1] == 'beth cook':
        teamrank.iloc[i, 1] = 'bethune cookman'
    if teamrank.iloc[i, 1] == 'sac state':
        teamrank.iloc[i, 1] = 'sacramento st'
    if teamrank.iloc[i, 1] == 'siu edward':
        teamrank.iloc[i, 1] = 'southern illinois'
    if teamrank.iloc[i, 1] == 'youngs st':
        teamrank.iloc[i, 1] = 'youngstown st'
    if teamrank.iloc[i, 1] == 'nw state':
        teamrank.iloc[i, 1] = 'northwestern st'
    if teamrank.iloc[i, 1] == 'cal st nrdge':
        teamrank.iloc[i, 1] = 'cal st northridge'
    if teamrank.iloc[i, 1] == 'ark pine bl':
        teamrank.iloc[i, 1] = 'arkansas pine bluff'
    if teamrank.iloc[i, 1] == 'va military':
        teamrank.iloc[i, 1] = 'vmi'
    if teamrank.iloc[i, 1] == 'incar word':
        teamrank.iloc[i, 1] = 'incarnate word'
    if teamrank.iloc[i, 1] == 'n arizona':
        teamrank.iloc[i, 1] = 'northern arizona' 
    if teamrank.iloc[i, 1] == 's car state':
        teamrank.iloc[i, 1] = 'south carolina state'
    if teamrank.iloc[i, 1] == 'nw st':
        teamrank.iloc[i, 1] = 'northwestern st'
    if teamrank.iloc[i, 1] == 'miss val st':
        teamrank.iloc[i, 1] = 'miss valley st'
    if teamrank.iloc[i, 1] == 'maryland es':
        teamrank.iloc[i, 1] = 'umes'
    if teamrank.iloc[i, 1] == 'alab a&m':
        teamrank.iloc[i, 1] = 'alabama a&m' 

In [39]:
teamrank_teams = pd.merge(teamrank, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(teamrank_teams)

,Rank,Team,Rating,Season,TeamNameSpelling,TeamID


The datasets joined perfectly.

In [40]:
# Need to drop duplicates because the spellings dataset had multiple rows for each team
teamrank_teams = teamrank_teams.drop(columns = ['Rank', 'TeamNameSpelling', 'Team']).drop_duplicates()
teamrank_teams = teamrank_teams.rename(index = str, columns = {'Rating': 'Teamrank_Rating'})

In [41]:
teamrank_teams.head()

,Teamrank_Rating,Season,TeamID
0,32.4,2008,1242
1,29.4,2008,1314
3,28.7,2008,1272
4,28.5,2008,1181
5,28.2,2008,1417


In [42]:
teamrank_teams.tail()

,Teamrank_Rating,Season,TeamID
5866,-17.6,2018,1152
5869,-18.1,2018,1290
5870,-19.2,2018,1271
5871,-19.9,2018,1175
5874,-20.3,2018,1105


In [43]:
teamrank10_teams = pd.merge(teamrank10, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(teamrank10_teams)

,Rank,Team,Rating,Season,TeamNameSpelling,TeamID
1,2,w virginia,32.6,2008,NaN,NaN
4,5,n carolina,28.7,2008,NaN,NaN
20,21,wash state,23.4,2008,NaN,NaN
45,40,st marys,19.7,2008,NaN,NaN
58,50,s carolina,18.2,2008,NaN,NaN
73,60,ga tech,17.1,2008,NaN,NaN
75,62,s mississippi,16.9,2008,NaN,NaN
85,69,va tech,15.7,2008,NaN,NaN
86,70,u mass,15.5,2008,NaN,NaN
89,72,s alabama,15.2,2008,NaN,NaN


I'll have to change the team names for the Teamrank last 10 games dataset too.

In [44]:
# For loop to change team names above
for i in range(len(teamrank10.Team)):
    if teamrank10.iloc[i, 1] == 'st marys':
        teamrank10.iloc[i, 1] = 'st marys ca'
    if teamrank10.iloc[i, 1] == 'wins salem':
        teamrank10.iloc[i, 1] = 'winston salem'
    if teamrank10.iloc[i, 1] == 'w virginia':
        teamrank10.iloc[i, 1] = 'west virginia'
    if teamrank10.iloc[i, 1] == 'n carolina':
        teamrank10.iloc[i, 1] = 'north carolina'
    if teamrank10.iloc[i, 1] == 'tx christian':
        teamrank10.iloc[i, 1] = 'tcu'
    if teamrank10.iloc[i, 1] == 'va tech':
        teamrank10.iloc[i, 1] = 'virginia tech'
    if teamrank10.iloc[i, 1] == 'miss state':
        teamrank10.iloc[i, 1] = 'mississippi st'
    if teamrank10.iloc[i, 1] == 'st bonavent':
        teamrank10.iloc[i, 1] = 'st bonaventure'
    if teamrank10.iloc[i, 1] == 'loyola chi':
        teamrank10.iloc[i, 1] = 'loyola chicago'
    if teamrank10.iloc[i, 1] == 's methodist':
        teamrank10.iloc[i, 1] = 'smu'
    if teamrank10.iloc[i, 1] == 'n mex state':
        teamrank10.iloc[i, 1] = 'new mexico st'
    if teamrank10.iloc[i, 1] == 's carolina':
        teamrank10.iloc[i, 1] = 'south carolina'
    if teamrank10.iloc[i, 1] == 'boston col':
        teamrank10.iloc[i, 1] = 'boston college'
    if teamrank10.iloc[i, 1] == 'e tenn st':
        teamrank10.iloc[i, 1] = 'etsu'
    if teamrank10.iloc[i, 1] == 'nc grnsboro':
        teamrank10.iloc[i, 1] = 'unc greensboro'
    if teamrank10.iloc[i, 1] == 'central fl':
        teamrank10.iloc[i, 1] = 'ucf'
    if teamrank10.iloc[i, 1] == 'utah val st':
        teamrank10.iloc[i, 1] = 'utah valley st'
    if teamrank10.iloc[i, 1] == 'northeastrn':
        teamrank10.iloc[i, 1] = 'northeastern'
    if teamrank10.iloc[i, 1] == 'ga tech':
        teamrank10.iloc[i, 1] = 'georgia tech'
    if teamrank10.iloc[i, 1] == 'col charlestn':
        teamrank10.iloc[i, 1] = 'college of charleston'
    if teamrank10.iloc[i, 1] == 'st josephs':
        teamrank10.iloc[i, 1] = 'st josephs pa'
    if teamrank10.iloc[i, 1] == 'u penn':
        teamrank10.iloc[i, 1] = 'penn'
    if teamrank10.iloc[i, 1] == 'ste f austin':
        teamrank10.iloc[i, 1] = 'stephen f austin'
    if teamrank10.iloc[i, 1] == 'fla gulf cst':
        teamrank10.iloc[i, 1] = 'florida gulf coast'
    if teamrank10.iloc[i, 1] == 'grd canyon':
        teamrank10.iloc[i, 1] = 'grand canyon'
    if teamrank10.iloc[i, 1] == 'tx arlington':
        teamrank10.iloc[i, 1] = 'ut arlington'
    if teamrank10.iloc[i, 1] == 'n iowa':
        teamrank10.iloc[i, 1] = 'northern iowa'
    if teamrank10.iloc[i, 1] == 'la tech':
        teamrank10.iloc[i, 1] = 'louisiana tech'
    if teamrank10.iloc[i, 1] == 'wm & mary':
        teamrank10.iloc[i, 1] = 'william & mary'
    if teamrank10.iloc[i, 1] == 'jksnville st':
        teamrank10.iloc[i, 1] = 'jacksonville st'
    if teamrank10.iloc[i, 1] == 'app state':
        teamrank10.iloc[i, 1] = 'appalachian st'
    if teamrank10.iloc[i, 1] == 'san fransco':
        teamrank10.iloc[i, 1] = 'san francisco'
    if teamrank10.iloc[i, 1] == 'e washingtn':
        teamrank10.iloc[i, 1] = 'eastern washington'
    if teamrank10.iloc[i, 1] == 'geo wshgtn':
        teamrank10.iloc[i, 1] = 'george washington'
    if teamrank10.iloc[i, 1] == 'u mass':
        teamrank10.iloc[i, 1] = 'umass'
    if teamrank10.iloc[i, 1] == 'maryland bc':
        teamrank10.iloc[i, 1] = 'umbc'
    if teamrank10.iloc[i, 1] == 'wash state':
        teamrank10.iloc[i, 1] = 'washington st'
    if teamrank10.iloc[i, 1] == 'tx san ant':
        teamrank10.iloc[i, 1] = 'utsa'
    if teamrank10.iloc[i, 1] == 'st fran pa':
        teamrank10.iloc[i, 1] = 'st francis pa'
    if teamrank10.iloc[i, 1] == 'miami oh':
        teamrank10.iloc[i, 1] = 'miami ohio'
    if teamrank10.iloc[i, 1] == 'geo mason':
        teamrank10.iloc[i, 1] = 'george mason'
    if teamrank10.iloc[i, 1] == 'wi milwkee':
        teamrank10.iloc[i, 1] = 'milwaukee'
    if teamrank10.iloc[i, 1] == 'tn state':
        teamrank10.iloc[i, 1] = 'tennessee st'
    if teamrank10.iloc[i, 1] == 'tn tech':
        teamrank10.iloc[i, 1] = 'tennessee tech'
    if teamrank10.iloc[i, 1] == 'nc wilmgton':
        teamrank10.iloc[i, 1] = 'unc wilmington'
    if teamrank10.iloc[i, 1] == 's alabama':
        teamrank10.iloc[i, 1] = 'south alabama'
    if teamrank10.iloc[i, 1] == 'lg beach st':
        teamrank10.iloc[i, 1] = 'long beach st'
    if teamrank10.iloc[i, 1] == 'james mad':
        teamrank10.iloc[i, 1] = 'james madison'
    if teamrank10.iloc[i, 1] == 'sam hous st':
        teamrank10.iloc[i, 1] = 'sam houston st'
    if teamrank10.iloc[i, 1] == 'cs bakersfld':
        teamrank10.iloc[i, 1] = 'cal state bakersfield'
    if teamrank10.iloc[i, 1] == 'loyola mymt':
        teamrank10.iloc[i, 1] = 'loyola marymount'
    if teamrank10.iloc[i, 1] == 's mississippi':
        teamrank10.iloc[i, 1] = 'southern miss'
    if teamrank10.iloc[i, 1] == 'bowling grn':
        teamrank10.iloc[i, 1] = 'bowling green'
    if teamrank10.iloc[i, 1] == 'tx el paso':
        teamrank10.iloc[i, 1] = 'utep'
    if teamrank10.iloc[i, 1] == 'n hampshire':
        teamrank10.iloc[i, 1] = 'new hampshire'
    if teamrank10.iloc[i, 1] == 'rob morris':
        teamrank10.iloc[i, 1] = 'robert morris'
    if teamrank10.iloc[i, 1] == 'wi grn bay':
        teamrank10.iloc[i, 1] = 'green bay'
    if teamrank10.iloc[i, 1] == 'charl south':
        teamrank10.iloc[i, 1] = 'charleston southern'
    if teamrank10.iloc[i, 1] == 'abl christian':
        teamrank10.iloc[i, 1] = 'abilene christian'
    if teamrank10.iloc[i, 1] == 'gard webb':
        teamrank10.iloc[i, 1] = 'gardner webb'
    if teamrank10.iloc[i, 1] == 'tx pan am':
        teamrank10.iloc[i, 1] = 'texas pan american'
    if teamrank10.iloc[i, 1] == 'se missouri':
        teamrank10.iloc[i, 1] = 'se missouri st'
    if teamrank10.iloc[i, 1] == 'neb omaha':
        teamrank10.iloc[i, 1] = 'omaha'
    if teamrank10.iloc[i, 1] == 's florida':
        teamrank10.iloc[i, 1] = 'south florida'
    if teamrank10.iloc[i, 1] == 'mass lowell':
        teamrank10.iloc[i, 1] = 'umass lowell'
    if teamrank10.iloc[i, 1] == 'e carolina':
        teamrank10.iloc[i, 1] = 'east carolina'
    if teamrank10.iloc[i, 1] == 'tx a&m cc':
        teamrank10.iloc[i, 1] = 'a&m corpus chris'
    if teamrank10.iloc[i, 1] == 's utah':
        teamrank10.iloc[i, 1] = 'southern utah'
    if teamrank10.iloc[i, 1] == 'n florida':
        teamrank10.iloc[i, 1] = 'north florida'
    if teamrank10.iloc[i, 1] == 'sacred hrt':
        teamrank10.iloc[i, 1] = 'sacred heart'
    if teamrank10.iloc[i, 1] == 'st fran ny':
        teamrank10.iloc[i, 1] = 'st francis ny'
    if teamrank10.iloc[i, 1] == 'ar lit rock':
        teamrank10.iloc[i, 1] = 'arkansas little rock'
    if teamrank10.iloc[i, 1] == 'beth cook':
        teamrank10.iloc[i, 1] = 'bethune cookman'
    if teamrank10.iloc[i, 1] == 'sac state':
        teamrank10.iloc[i, 1] = 'sacramento st'
    if teamrank10.iloc[i, 1] == 'siu edward':
        teamrank10.iloc[i, 1] = 'southern illinois'
    if teamrank10.iloc[i, 1] == 'youngs st':
        teamrank10.iloc[i, 1] = 'youngstown st'
    if teamrank10.iloc[i, 1] == 'nw state':
        teamrank10.iloc[i, 1] = 'northwestern st'
    if teamrank10.iloc[i, 1] == 'cal st nrdge':
        teamrank10.iloc[i, 1] = 'cal st northridge'
    if teamrank10.iloc[i, 1] == 'ark pine bl':
        teamrank10.iloc[i, 1] = 'arkansas pine bluff'
    if teamrank10.iloc[i, 1] == 'va military':
        teamrank10.iloc[i, 1] = 'vmi'
    if teamrank10.iloc[i, 1] == 'incar word':
        teamrank10.iloc[i, 1] = 'incarnate word'
    if teamrank10.iloc[i, 1] == 'n arizona':
        teamrank10.iloc[i, 1] = 'northern arizona' 
    if teamrank10.iloc[i, 1] == 's car state':
        teamrank10.iloc[i, 1] = 'south carolina state'
    if teamrank10.iloc[i, 1] == 'nw st':
        teamrank10.iloc[i, 1] = 'northwestern st'
    if teamrank10.iloc[i, 1] == 'miss val st':
        teamrank10.iloc[i, 1] = 'miss valley st'
    if teamrank10.iloc[i, 1] == 'maryland es':
        teamrank10.iloc[i, 1] = 'umes'
    if teamrank10.iloc[i, 1] == 'alab a&m':
        teamrank10.iloc[i, 1] = 'alabama a&m' 

In [45]:
teamrank10_teams = pd.merge(teamrank10, spellings, how = 'left', left_on = 'Team', right_on = 'TeamNameSpelling')
nans(teamrank10_teams)

,Rank,Team,Rating,Season,TeamNameSpelling,TeamID


The datasets joined perfectly.

In [46]:
# Need to drop duplicates because the spellings dataset had multiple rows for each team
teamrank10_teams = teamrank10_teams.drop(columns = ['Rank', 'TeamNameSpelling', 'Team']).drop_duplicates()
teamrank10_teams = teamrank10_teams.rename(index = str, columns = {'Rating': 'Teamrank10_Rating'})

In [47]:
teamrank10_teams.head()

,Teamrank10_Rating,Season,TeamID
0,33.9,2008,1181
1,32.6,2008,1452
3,31.8,2008,1242
4,29.3,2008,1417
5,28.7,2008,1314


In [48]:
teamrank10_teams.tail()

,Teamrank10_Rating,Season,TeamID
5342,-18.8,2018,1322
5344,-19.3,2018,1136
5345,-19.9,2018,1175
5348,-22.8,2018,1271
5349,-27.2,2018,1105


In [49]:
trank_teams = pd.merge(trank, spellings, how = 'left', left_on = 'TEAM', right_on = 'TeamNameSpelling')
nans(trank_teams)

,RK,TEAM,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,TOR,TORD,ORB,...,FTRD,2P%,2P%D,3P%,3P%D,ADJ T.,WAB,Season,TeamNameSpelling,TeamID
422,263,ut rio grande valley,102.8,112.8,0.2554,53.8,54.6,19.0,22.6,27.6,...,34.5,50.4,55.8,39.6,34.8,64.5,-9.6,2008,NaN,NaN
432,269,texas a&m corpus chris,94.9,104.8,0.2433,46.4,48.0,27.1,19.3,39.6,...,45.2,48.0,44.3,28.8,36.6,65.8,-14.5,2008,NaN,NaN
486,300,southeast missouri st,94.6,109.3,0.1597,50.2,51.2,23.5,20.0,32.9,...,43.9,49.5,48.9,34.5,36.7,76.5,-14.6,2008,NaN,NaN
502,310,mississippi valley st,90.0,105.3,0.1404,44.3,49.7,21.8,22.9,32.5,...,37.5,41.5,50.9,35.2,31.3,64.9,-10.0,2008,NaN,NaN
513,316,cal st bakersfield,94.8,112.0,0.1279,46.0,55.6,19.4,19.7,34.1,...,31.8,43.3,55.5,34.4,37.2,67.2,-16.8,2008,NaN,NaN
528,326,winston salem st,88.2,105.9,0.1088,43.8,48.7,23.1,22.9,33.4,...,36.3,43.8,50.1,29.2,30.7,67.7,-14.1,2008,NaN,NaN
832,174,texas a&m corpus chris,96.7,97.9,0.4641,46.8,48.3,24.7,22.0,37.0,...,38.6,46.8,49.2,31.1,31.0,67.9,-8.0,2009,NaN,NaN
991,267,cal st bakersfield,97.9,107.5,0.2536,45.1,52.1,19.6,21.1,34.0,...,37.3,43.2,49.5,32.3,38.7,66.4,-14.5,2009,NaN,NaN
1085,324,ut rio grande valley,91.0,109.8,0.1034,48.6,54.1,23.1,21.3,28.6,...,39.5,50.4,53.3,29.5,37.0,64.4,-13.1,2009,NaN,NaN
1090,329,mississippi valley st,91.2,111.4,0.0901,43.6,54.7,21.8,22.2,33.0,...,52.2,41.1,55.7,32.4,35.2,68.6,-18.3,2009,NaN,NaN


Some of the team names in the Trank Ratings were different from the names in the Kaggle Dataset, therefore I must change the names in the Trank Ratings so I can merge the two datasets together.

In [50]:
# For loop to change team names above
for i in range(len(trank.TEAM)):
    if trank.iloc[i, 1] == 'winston salem st':
        trank.iloc[i, 1] = 'winston salem'
    if trank.iloc[i, 1] == 'cal st bakersfield':
        trank.iloc[i, 1] = 'cal state bakersfield'
    if trank.iloc[i, 1] == 'southeast missouri st':
        trank.iloc[i, 1] = 'se missouri st'
    if trank.iloc[i, 1] == 'ut rio grande valley':
        trank.iloc[i, 1] = 'texas rio grande valley'
    if trank.iloc[i, 1] == 'texas a&m corpus chris':
        trank.iloc[i, 1] = 'a&m corpus chris'
    if trank.iloc[i, 1] == 'mississippi valley st':
        trank.iloc[i, 1] = 'miss valley st'

In [51]:
trank_teams = pd.merge(trank, spellings, how = 'left', left_on = 'TEAM', right_on = 'TeamNameSpelling')
nans(trank_teams)

,RK,TEAM,ADJOE,ADJDE,BARTHAG,EFG%,EFGD%,TOR,TORD,ORB,...,FTRD,2P%,2P%D,3P%,3P%D,ADJ T.,WAB,Season,TeamNameSpelling,TeamID


The datasets joined perefectly.

In [52]:
# Need to drop duplicates because the spellings dataset had multiple rows for each team
trank_teams = trank_teams.drop(columns = ['RK', 'WAB', 'TeamNameSpelling', 'TEAM']).drop_duplicates()
trank_teams = trank_teams.rename(index = str, columns = {'BARTHAG': 'Trank_Rating', 'ADJOE': 'Trank_OE', 'ADJDE': 'Trank_DE', 'ADJ T.': 'Trank_Tempo'})

In [53]:
trank_teams.head()

,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,TOR,TORD,ORB,DRB,FTR,FTRD,2P%,2P%D,3P%,3P%D,Trank_Tempo,Season,TeamID
0,121.0,85.6,0.9816,56.3,44.8,18.7,22.9,38.0,29.0,37.5,30.8,54.8,40.9,39.9,34.0,69.5,2008,1242
1,113.3,83.8,0.9697,53.0,42.5,17.2,23.3,37.8,29.3,38.6,32.7,53.3,41.6,35.0,29.8,70.7,2008,1272
2,116.4,86.9,0.9664,52.3,48.0,18.8,22.7,39.2,25.9,38.0,25.7,52.9,46.8,33.8,34.2,66.2,2008,1417
3,112.2,84.9,0.9609,50.6,43.1,19.4,22.2,36.2,28.5,39.3,25.4,49.2,41.3,35.9,31.1,63.5,2008,1458
4,117.2,88.8,0.9607,54.1,47.6,18.2,24.9,34.3,33.5,40.4,32.0,51.8,47.1,38.4,32.8,73.7,2008,1181


In [54]:
trank_teams.tail()

,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,TOR,TORD,ORB,DRB,FTR,FTRD,2P%,2P%D,3P%,3P%D,Trank_Tempo,Season,TeamID
6265,85.9,109.6,0.0574,43.3,49.2,20.5,17.9,22.3,31.9,27.1,37.6,41.0,48.3,30.5,33.7,68.7,2018,1164
6268,89.2,114.8,0.0520,42.6,54.9,19.1,19.6,30.4,31.1,29.1,39.2,41.6,54.6,30.1,36.8,69.9,2018,1290
6269,90.9,117.7,0.0489,43.1,54.7,19.4,16.0,28.2,34.7,29.7,31.8,44.3,54.3,27.5,36.8,66.4,2018,1271
6271,87.4,114.1,0.0446,45.0,54.0,24.2,16.6,30.0,32.5,31.4,29.8,44.8,52.3,30.3,38.4,67.5,2018,1105
6272,90.3,119.9,0.0368,47.3,55.9,22.8,18.2,29.3,33.7,28.5,35.7,46.1,53.8,33.2,38.9,66.2,2018,1175


In [55]:
kenpom_teams = pd.merge(kenpom, spellings, how = 'left', left_on = 'TeamName', right_on = 'TeamNameSpelling')
kenpom_teams[kenpom_teams['TeamID'].isnull()]

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed,TeamNameSpelling,TeamID
431,2008,texas a&m corpus chris,66.7343,183,62.9549,232,95.2254,275,96.6362,270,104.3210,226,106.8860,233,-10.249400,270,NaN,NaN,NaN
494,2008,mississippi valley st,65.0411,252,62.3905,258,93.0497,306,92.1127,322,101.0450,142,107.0800,242,-14.967400,304,16.0,NaN,NaN
501,2008,southeast missouri st,75.0078,7,73.7425,3,98.8361,221,95.9771,278,107.8740,293,111.5190,316,-15.541600,310,NaN,NaN,NaN
515,2008,cal st bakersfield,67.3484,161,64.4884,167,97.7281,239,96.4741,272,110.7590,329,113.5400,327,-17.066300,319,NaN,NaN,NaN
521,2008,winston salem st,68.1908,129,64.9701,148,90.0764,328,89.6180,329,99.1002,96,107.7360,250,-18.117500,323,NaN,NaN,NaN
610,2009,cal st bakersfield,65.6643,200,64.0979,189,98.5392,222,99.7587,221,108.8590,317,109.3550,299,-9.595810,272,NaN,NaN,NaN
820,2009,mississippi valley st,68.7621,90,65.6783,121,92.5380,304,93.6731,296,108.3910,312,112.3180,327,-18.644900,323,NaN,NaN,NaN
979,2009,southeast missouri st,70.8871,38,69.0381,30,95.2156,275,94.4852,288,115.5810,344,116.5380,343,-22.053300,336,NaN,NaN,NaN
1019,2009,texas a&m corpus chris,68.8340,87,65.7566,118,96.3456,266,99.5689,224,98.2566,101,99.2890,101,0.279812,160,NaN,NaN,NaN
1101,2009,winston salem st,67.9707,112,66.6691,88,83.1568,343,83.5374,343,99.9208,137,107.4600,273,-23.922800,338,NaN,NaN,NaN


Some of the team names in the Kenpom Ratings were different from the names in the Kaggle Dataset, therefore I'll change the names in the Kenpom Ratings so I can merge the two datasets together.

In [56]:
# For loop to change team names above
for i in range(len(kenpom.TeamName)):
    if kenpom.iloc[i, 1] == 'winston salem st':
        kenpom.iloc[i, 1] = 'winston salem'
    if kenpom.iloc[i, 1] == 'cal st bakersfield':
        kenpom.iloc[i, 1] = 'cal state bakersfield'
    if kenpom.iloc[i, 1] == 'mississippi valley st':
        kenpom.iloc[i, 1] = 'miss valley st'
    if kenpom.iloc[i, 1] == 'southeast missouri st':
        kenpom.iloc[i, 1] = 'se missouri st'
    if kenpom.iloc[i, 1] == 'texas a&m corpus chris':
        kenpom.iloc[i, 1] = 'a&m corpus chris'
    if kenpom.iloc[i, 1] == 'ut rio grande valley':
        kenpom.iloc[i, 1] = 'texas rio grande valley'

In [57]:
kenpom_teams = pd.merge(kenpom, spellings, how = 'left', left_on = 'TeamName', right_on = 'TeamNameSpelling')
kenpom_teams[kenpom_teams['TeamID'].isnull()]

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM,seed,TeamNameSpelling,TeamID


In [58]:
# Need to drop duplicates because the spellings dataset had multiple rows for each team
kenpom_teams = kenpom_teams.drop(columns = ['Tempo', 'RankTempo', 'RankAdjTempo', 'OE', 'RankOE', 'RankAdjOE', 'DE', 'RankDE', 'RankAdjDE', 'RankAdjEM', 'TeamNameSpelling']).drop_duplicates()
kenpom_teams = kenpom_teams.rename(index = str, columns = {'TeamName': 'Team', 'AdjTempo': 'Kenpom_Tempo', 'AdjOE': 'Kenpom_OE', 'AdjDE': 'Kenpom_DE', 'AdjEM': 'Kenpom_Rating'})

In [59]:
kenpom_teams.head()

,Season,Team,Kenpom_Tempo,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed,TeamID
0,2008,kansas,66.8546,121.433,87.4681,33.9647,1.0,1242
1,2008,ucla,63.1761,118.735,88.1600,30.5753,1.0,1417
2,2008,memphis,67.7113,114.695,85.3169,29.3779,1.0,1272
3,2008,north carolina,71.9191,120.686,92.5415,28.1446,1.0,1314
5,2008,duke,70.9709,117.443,89.8149,27.6285,2.0,1181


In [60]:
kenpom_teams.tail()

,Season,Team,Kenpom_Tempo,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed,TeamID
6264,2018,wright st,68.2124,101.068,98.2510,2.81674,14.0,1460
6267,2018,wyoming,73.5044,105.921,100.8070,5.11463,NaN,1461
6268,2018,xavier,70.4307,120.740,99.0492,21.69090,1.0,1462
6269,2018,yale,68.7887,104.050,107.2290,-3.17937,NaN,1463
6270,2018,youngstown st,71.0824,101.405,116.4890,-15.08350,NaN,1464


Now we can merge all efficiency metric datasets together.

In [61]:
teams = pd.merge(colley_teams, teamrank_teams, on = ['TeamID', 'Season'])
teams = pd.merge(teams, teamrank10_teams, on = ['TeamID', 'Season'])
teams = pd.merge(teams, trank_teams, on = ['TeamID', 'Season'])
teams = pd.merge(teams, kenpom_teams, on = ['TeamID', 'Season'])

In [62]:
teams.head()

,Colley_Rating,Season,TeamID,Teamrank_Rating,Teamrank10_Rating,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,...,2P%D,3P%,3P%D,Trank_Tempo,Team,Kenpom_Tempo,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed
0,1.08474,2008,1314,29.4,28.7,120.2,91.8,0.9572,52.4,48.0,...,47.0,37.8,33.2,75.1,north carolina,71.9191,120.686,92.5415,28.1446,1.0
1,1.04691,2008,1272,28.7,26.7,113.3,83.8,0.9697,53.0,42.5,...,41.6,35.0,29.8,70.7,memphis,67.7113,114.695,85.3169,29.3779,1.0
2,1.03847,2008,1417,28.2,29.3,116.4,86.9,0.9664,52.3,48.0,...,46.8,33.8,34.2,66.2,ucla,63.1761,118.735,88.1600,30.5753,1.0
3,1.03505,2008,1397,25.7,26.6,116.3,92.2,0.9353,52.8,49.2,...,50.8,36.4,31.0,72.7,tennessee,69.5841,117.787,95.4494,22.3372,2.0
4,1.01374,2008,1242,32.4,31.8,121.0,85.6,0.9816,56.3,44.8,...,40.9,39.9,34.0,69.5,kansas,66.8546,121.433,87.4681,33.9647,1.0


#### Creating a Weighted Rating

Now we want to be able to come up with one rating to determine if a team is "better" than another. It will be a combination of each rating system, weighting the results using a ridge regression on prior NCAA tournament results. We want to use ridge regression because the efficiency ratings are naturlly correlated and we don't want to completely factor out any of our metrics.

To do this, we will first scale each rating with a minmax scale so each rating value is between 0 to 1. This is so we can see which rating system has a bigger influence on tournament results. And we'll do this scaling after separating by year, so we can do it with only 2019 data later.

In [63]:
teams_2008 = teams.query('Season == 2008')
teams_2010 = teams.query('Season == 2010')
teams_2011 = teams.query('Season == 2011')
teams_2012 = teams.query('Season == 2012')
teams_2013 = teams.query('Season == 2013')
teams_2014 = teams.query('Season == 2014')
teams_2015 = teams.query('Season == 2015')
teams_2016 = teams.query('Season == 2016')
teams_2017 = teams.query('Season == 2017')
teams_2018 = teams.query('Season == 2018')

In [64]:
scale = MinMaxScaler()
scale_columns = ['Colley_Rating', 'Teamrank_Rating', 'Teamrank10_Rating', 'Trank_Rating', 'Kenpom_Rating']
pd.options.mode.chained_assignment = None
teams_2008[scale_columns] = scale.fit_transform(teams_2008[scale_columns])
teams_2010[scale_columns] = scale.fit_transform(teams_2010[scale_columns])
teams_2011[scale_columns] = scale.fit_transform(teams_2011[scale_columns])
teams_2012[scale_columns] = scale.fit_transform(teams_2012[scale_columns])
teams_2013[scale_columns] = scale.fit_transform(teams_2013[scale_columns])
teams_2014[scale_columns] = scale.fit_transform(teams_2014[scale_columns])
teams_2015[scale_columns] = scale.fit_transform(teams_2015[scale_columns])
teams_2016[scale_columns] = scale.fit_transform(teams_2016[scale_columns])
teams_2017[scale_columns] = scale.fit_transform(teams_2017[scale_columns])
teams_2018[scale_columns] = scale.fit_transform(teams_2018[scale_columns])
pd.options.mode.chained_assignment = 'warn'

In [65]:
teams = pd.concat([teams_2008, teams_2010, teams_2011, teams_2012, teams_2013, teams_2014, teams_2015, teams_2016, teams_2017, teams_2018])

In [66]:
ranking_data = teams[['Season', 'TeamID', 'Colley_Rating', 'Teamrank_Rating', 'Teamrank10_Rating', 'Trank_Rating', 'Kenpom_Rating']]

In [67]:
ranking_data.head()

,Season,TeamID,Colley_Rating,Teamrank_Rating,Teamrank10_Rating,Trank_Rating,Kenpom_Rating
0,2008,1314,1.000000,0.939394,0.903704,0.974641,0.912825
1,2008,1272,0.967124,0.925253,0.866667,0.987633,0.931297
2,2008,1417,0.959789,0.915152,0.914815,0.984203,0.949233
3,2008,1397,0.956817,0.864646,0.864815,0.951881,0.825840
4,2008,1242,0.938298,1.000000,0.961111,1.000000,1.000000


In [68]:
ranking_data.to_csv('mydata/ranking_data.csv', index = False)

After exporting this ranking data to a csv file, I perform linear regression in a separate notebook (see RatingLinearRegression.ipynb).

The weights below are those we obtained from the ridge regression.

In [69]:
weights = [6.71236012, 11.19605646, 3.11732104, 13.51627166, 8.3058081]

In [70]:
teams['Weighted_Rating'] = weights[0] * teams['Colley_Rating'] + weights[1] * teams['Teamrank_Rating'] + weights[2] * teams['Teamrank10_Rating'] + weights[3] * teams['Trank_Rating'] + weights[4] * teams['Kenpom_Rating']

In [71]:
teams.head()

,Colley_Rating,Season,TeamID,Teamrank_Rating,Teamrank10_Rating,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,...,3P%,3P%D,Trank_Tempo,Team,Kenpom_Tempo,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed,Weighted_Rating
0,1.000000,2008,1314,0.939394,0.903704,120.2,91.8,0.974641,52.4,48.0,...,37.8,33.2,75.1,north carolina,71.9191,120.686,92.5415,0.912825,1.0,40.802268
1,0.967124,2008,1272,0.925253,0.866667,113.3,83.8,0.987633,53.0,42.5,...,35.0,29.8,70.7,memphis,67.7113,114.695,85.3169,0.931297,1.0,40.636830
2,0.959789,2008,1417,0.915152,0.914815,116.4,86.9,0.984203,52.3,48.0,...,33.8,34.2,66.2,ucla,63.1761,118.735,88.1600,0.949233,1.0,40.727208
3,0.956817,2008,1397,0.864646,0.864815,116.3,92.2,0.951881,52.8,49.2,...,36.4,31.0,72.7,tennessee,69.5841,117.787,95.4494,0.825840,2.0,38.524187
4,0.938298,2008,1242,1.000000,0.961111,121.0,85.6,1.000000,56.3,44.8,...,39.9,34.0,69.5,kansas,66.8546,121.433,87.4681,1.000000,1.0,42.312421


In [72]:
teams = teams.dropna(subset = ['seed'])  # Drop teams without a seed, aka teams who did not make the tournament
teams = teams.drop(columns = ['Team'])
teams.tail()

,Colley_Rating,Season,TeamID,Teamrank_Rating,Teamrank10_Rating,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,...,2P%D,3P%,3P%D,Trank_Tempo,Kenpom_Tempo,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed,Weighted_Rating
3291,0.522386,2018,1347,0.434483,0.560311,101.3,102.8,0.451499,49.5,49.4,...,48.4,35.1,33.9,64.1,62.7351,103.5810,103.629,0.469744,16.0,20.121795
3293,0.517718,2018,1233,0.542529,0.570039,108.7,107.2,0.539722,54.1,52.1,...,51.4,38.8,35.5,71.6,70.6965,110.2610,107.388,0.517835,15.0,22.922351
3383,0.365244,2018,1411,0.354023,0.441634,103.7,111.7,0.280086,50.6,50.9,...,49.1,36.0,36.4,72.5,71.5742,105.2810,112.940,0.344408,16.0,14.438323
3399,0.347387,2018,1254,0.324138,0.459144,101.4,110.1,0.259529,52.1,50.0,...,50.5,34.6,32.8,72.8,71.3779,103.3350,111.135,0.342081,16.0,13.741275
3428,0.297169,2018,1300,0.278161,0.361868,95.7,110.5,0.133084,50.4,48.5,...,48.1,32.3,32.8,66.4,64.9456,98.1525,110.889,0.260793,16.0,10.201960


### Regular Season Stats

Kaggle provides a csv file of the regular season results for all torunament teams since 2003. I will use this dataset to get the remaining offensive and defensive stats I need for my analysis.

In [73]:
reg_season = pd.read_csv('DataFiles/RegularSeasonDetailedResults.csv')

In [74]:
reg_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [75]:
reg_season = reg_season.query('Season > 2009 | Season == 2008')  # because these are the years we have efficiency data

In [76]:
my_data = reg_season[['Season', 'WTeamID', 'LTeamID', 'WFGM', 'LFGM', 'WFGA', 'LFGA', 'WFGA3', 'LFGA3', 'WFTM', 'LFTM', 'WFTA', 'LFTA', 'WAst', 'LAst']]

In [77]:
# w_data is data for games in which the team won, and l_data is data for the games in which the team lost
w_data = my_data.groupby(['Season', 'WTeamID']).sum().drop(columns = ['LTeamID']).reset_index()
l_data = my_data.groupby(['Season', 'LTeamID']).sum().drop(columns = ['WTeamID']).reset_index()

In [78]:
w_data.head()

,Season,WTeamID,WFGM,LFGM,WFGA,LFGA,WFGA3,LFGA3,WFTM,LFTM,WFTA,LFTA,WAst,LAst
0,2008,1102,285,244,587,672,274,295,202,132,286,203,197,144
1,2008,1103,577,493,1216,1165,487,374,345,280,486,412,328,252
2,2008,1104,504,413,1038,1026,316,384,240,224,389,319,293,204
3,2008,1105,244,224,616,617,197,175,228,156,316,238,113,119
4,2008,1106,459,362,947,934,274,272,238,199,354,340,265,173


In [79]:
l_data.head()

,Season,LTeamID,WFGM,LFGM,WFGA,LFGA,WFGA3,LFGA3,WFTM,LFTM,WFTA,LFTA,WAst,LAst
0,2008,1102,291,234,643,600,292,256,191,173,262,254,194,137
1,2008,1103,257,210,513,558,156,230,150,156,216,222,135,116
2,2008,1104,425,413,901,972,343,324,246,178,335,285,248,222
3,2008,1105,382,315,811,853,227,277,248,167,341,259,217,137
4,2008,1106,259,249,576,602,169,213,172,91,239,155,142,126


In [80]:
wl_data = pd.merge(w_data, l_data, left_on = ['Season', 'WTeamID'], right_on = ['Season', 'LTeamID'], how = 'inner')

In [81]:
stats = pd.DataFrame()
stats['Season'] = wl_data['Season']
stats['TeamID'] = wl_data['WTeamID']
stats['3ptRate'] = (wl_data['WFGA3_x'] + wl_data['LFGA3_y']) / (wl_data['WFGA_x'] + wl_data['LFGA_y'])
stats['Ast%'] = (wl_data['WAst_x'] + wl_data['LAst_y']) / (wl_data['WFGM_x'] + wl_data['LFGM_y'])
stats['FT%'] = (wl_data['WFTM_x'] + wl_data['LFTM_y']) / (wl_data['WFTA_x'] + wl_data['LFTA_y'])
stats['Opp3ptRate'] = (wl_data['WFGA3_y'] + wl_data['LFGA3_x']) / (wl_data['WFGA_y'] + wl_data['LFGA_x'])
stats['OppAst%'] = (wl_data['WAst_y'] + wl_data['LAst_x']) / (wl_data['WFGM_y'] + wl_data['LFGM_x'])

In [82]:
stats.head()

,Season,TeamID,3ptRate,Ast%,FT%,Opp3ptRate,OppAst%
0,2008,1102,0.446504,0.643545,0.694444,0.446388,0.631776
1,2008,1103,0.404171,0.564168,0.707627,0.315852,0.516000
2,2008,1104,0.318408,0.561614,0.620178,0.377270,0.539379
3,2008,1105,0.322668,0.447227,0.686957,0.281513,0.554455
4,2008,1106,0.314396,0.552260,0.646365,0.292053,0.507246


In [83]:
teams = pd.merge(teams, stats, on = ['Season', 'TeamID'])

In [84]:
# To put the columns in decimal form
teams['FTR'] = teams['FTR'] / 100
teams['FTRD'] = teams['FTRD'] / 100
teams['2P%'] = teams['2P%'] / 100
teams['2P%D'] = teams['2P%D'] / 100
teams['3P%'] = teams['3P%'] / 100
teams['3P%D'] = teams['3P%D'] / 100

In [85]:
teams.head()

,Colley_Rating,Season,TeamID,Teamrank_Rating,Teamrank10_Rating,Trank_OE,Trank_DE,Trank_Rating,EFG%,EFGD%,...,Kenpom_OE,Kenpom_DE,Kenpom_Rating,seed,Weighted_Rating,3ptRate,Ast%,FT%,Opp3ptRate,OppAst%
0,1.000000,2008,1314,0.939394,0.903704,120.2,91.8,0.974641,52.4,48.0,...,120.686,92.5415,0.912825,1.0,40.802268,0.221826,0.525392,0.753986,0.347945,0.504329
1,0.967124,2008,1272,0.925253,0.866667,113.3,83.8,0.987633,53.0,42.5,...,114.695,85.3169,0.931297,1.0,40.636830,0.367143,0.557260,0.595561,0.287572,0.490489
2,0.959789,2008,1417,0.915152,0.914815,116.4,86.9,0.984203,52.3,48.0,...,118.735,88.1600,0.949233,1.0,40.727208,0.281216,0.542237,0.736091,0.278707,0.506667
3,0.956817,2008,1397,0.864646,0.864815,116.3,92.2,0.951881,52.8,49.2,...,117.787,95.4494,0.825840,2.0,38.524187,0.390556,0.615716,0.654206,0.356119,0.554855
4,0.938298,2008,1242,1.000000,0.961111,121.0,85.6,1.000000,56.3,44.8,...,121.433,87.4681,1.000000,1.0,42.312421,0.291796,0.627572,0.707756,0.380901,0.535411


In [86]:
teams.to_csv('mydata/teamsheets.csv', index = False)

The above file is data for each team in the NCAA tournament in 2008 and 20010-2018. This will be used in the notebook MatchupFeatureEngineering.ipynb